In [2]:
########################
# Constant declarations
########################

from math import exp
from math import factorial

gamma = 0.9
max_transfer = 5
max_cars = 20
theta = 0.01
num_states = (max_cars + 1) ** 2

# Expected values for Poisson random variables
lambda_a_rental = 3
lambda_b_rental = 4
lambda_a_return = 3
lambda_b_return = 2

value = [0] * num_states
policy = [0] * num_states
reward = [0] * num_states
prob_a = [0] * 21
prob_b = [0] * 21

########################
# Function declarations
########################


def policy_evaluation():  # Iterate the Bellman equation, done in place
    delta = 1
    while delta >= theta:  # Repeat until delta is less than theta
        delta = 0
        for s in range(num_states):  # Iterates through every state
            v = value[s]  # Preserves old value of state
            value[s] = bellman(s)
            delta = max(delta, abs(v - value[s]))  # Sets delta to the largest update


def policy_improvement():
    t = True
    action_value = [0] * 11
    for s in range(num_states):
        b = policy[s]
        for i in range(11):
            if i > 5:
                action_value[i] = bellman2(s, 5-i)
            else:
                action_value[i] = bellman2(s, i)
        policy[s] = policy_arg(action_value)
        if b != policy[s]:
            t = False  # policy_stable set to false
        elif b == policy[s] and t == False:
            t = False
        else:
            t = True  # policy_stable set to true
    return t


def policy_arg(l):
    index = 0
    v = l[0]
    for i in range(len(l)):
        if v < l[i]:
            index = i
            v = l[i]
    if index > 5:
        index = 5 - index
    return index


def reward_setting():
    for n in range(num_states):
        a = n // 21
        b = n % 21
        temp = 0
        for i in range(a+1):
            for j in range(b+1):
                temp += 10 * (i+j) * poisson(3, i) * poisson(4, j)
        reward[n] = temp


def prob_a_setting(a1):
    for i in range(21):
        prob_a[i] = 0
        temp = 0
        for j in range(a1+1):
            if i - a1 + j >= 0:
                temp += poisson(3, j) * poisson(3, i - a1 + j)
        prob_a[i] = temp
    t = 0
    for i in range(21):
        t += prob_a[i]
    for i in range(21):
        prob_a[i] = prob_a[i] / t


def prob_b_setting(b1):
    for i in range(21):
        prob_b[i] = 0
        temp = 0
        for j in range(b1+1):
            if i - b1 + j >= 0:
                temp += poisson(4, j) * poisson(2, i - b1 + j)
        prob_b[i] = temp
    t = 0
    for i in range(21):
        t += prob_b[i]
    for i in range(21):
        prob_b[i] = prob_b[i] / t


def bellman(s):
    a = s // 21
    b = s % 21

    num_transfer = get_policy(a, b)
    num_transfer = max(-b, min(num_transfer, a))  # Ensure num to transfer exists in source
    num_transfer = max(-max_transfer, min(num_transfer, max_transfer))  # Ensure transfer does not exceed max

    new_a = a - num_transfer
    new_b = b + num_transfer
    temp = 0

    prob_a_setting(new_a)
    prob_b_setting(new_b)

    for n in range(num_states):
        a2 = n // 21
        b2 = n % 21
        temp += prob_a[a2] * prob_b[b2] * (get_reward(new_a, new_b) + gamma * get_value(a2, b2))
        # Sum the component of Bellman
    temp -= abs(num_transfer) * 2
    if temp < 0:
        return 0
    return temp


def bellman2(s, p):
    a = s // 21
    b = s % 21

    num_transfer = p
    num_transfer = max(-b, min(num_transfer, a))  # Ensure num to transfer exists in source
    num_transfer = max(-max_transfer, min(num_transfer, max_transfer))  # Ensure transfer does not exceed max

    if a - num_transfer > 20:
        return 0
    if b + num_transfer > 20:
        return 0
    new_a = a - num_transfer
    new_b = b + num_transfer
    temp = 0

    prob_a_setting(new_a)
    prob_b_setting(new_b)
    for n in range(num_states):
        a2 = n // 21
        b2 = n % 21
        temp += prob_a[a2] * prob_b[b2] * (get_reward(new_a, new_b) + gamma * get_value(a2, b2))
        # Sum the component of Bellman
    temp -= abs(num_transfer) * 2
    if temp < 0:
        return 0
    return temp


def poisson(expected, n):
    return (expected ** n) * (exp(-expected)) / (factorial(n))  # Evaluates Poisson random variable


def get_value(a, b):
    return value[a * (max_cars+1) + b]


def get_policy(a, b):
    return policy[a * (max_cars+1) + b]


def get_reward(a, b):
    return reward[a * (max_cars+1) + b]


########################
reward_setting()
# for i in range(21):
#     print reward[(20 - i) * 21], reward[(20 - i) * 21 + 1], reward[(20 - i) * 21 + 2], reward[(20 - i) * 21 + 3], \
#         reward[(20 - i) * 21 + 4], reward[(20 - i) * 21 + 5], reward[(20 - i) * 21 + 6], reward[(20 - i) * 21 + 7], \
#         reward[(20 - i) * 21 + 8], reward[(20 - i) * 21 + 9], reward[(20 - i) * 21 + 10], \
#         reward[(20 - i) * 21 + 11], reward[(20 - i) * 21 + 12], reward[(20 - i) * 21 + 13], \
#         reward[(20 - i) * 21 + 14], reward[(20 - i) * 21 + 15], reward[(20 - i) * 21 + 16], \
#         reward[(20 - i) * 21 + 17], reward[(20 - i) * 21 + 18], reward[(20 - i) * 21 + 19], \
#         reward[(20 - i) * 21 + 20]
# print '-------------------------------------------------------------------------------'
policy_stable = False
while not policy_stable:
    policy_evaluation()
    policy_stable = policy_improvement()
    for i in range(21):
        print '%2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d %2d' % (
            policy[(20 - i) * 21], policy[(20 - i) * 21 + 1], policy[(20 - i) * 21 + 2], policy[(20 - i) * 21 + 3], \
            policy[(20 - i) * 21 + 4], policy[(20 - i) * 21 + 5], policy[(20 - i) * 21 + 6], policy[(20 - i) * 21 + 7],\
            policy[(20 - i) * 21 + 8], policy[(20 - i) * 21 + 9], policy[(20 - i) * 21 + 10],\
            policy[(20 - i) * 21 + 11], policy[(20 - i) * 21 + 12], policy[(20 - i) * 21 + 13],\
            policy[(20 - i) * 21 + 14], policy[(20 - i) * 21 + 15], policy[(20 - i) * 21 + 16],\
            policy[(20 - i) * 21 + 17], policy[(20 - i) * 21 + 18], policy[(20 - i) * 21 + 19], \
            policy[(20 - i) * 21 + 20])
    print '-------------------------------------------------------------------------------'
#     for i in range(21):
#         print value[(20 - i) * 21], value[(20 - i) * 21 + 1], value[(20 - i) * 21 + 2], value[(20 - i) * 21 + 3], \
#             value[(20 - i) * 21 + 4], value[(20 - i) * 21 + 5], value[(20 - i) * 21 + 6], value[(20 - i) * 21 + 7], \
#             value[(20 - i) * 21 + 8], value[(20 - i) * 21 + 9], value[(20 - i) * 21 + 10], \
#             value[(20 - i) * 21 + 11], value[(20 - i) * 21 + 12], value[(20 - i) * 21 + 13], \
#             value[(20 - i) * 21 + 14], value[(20 - i) * 21 + 15], value[(20 - i) * 21 + 16], \
#             value[(20 - i) * 21 + 17], value[(20 - i) * 21 + 18], value[(20 - i) * 21 + 19], \
#             value[(20 - i) * 21 + 20]
#     print '-------------------------------------------------------------------------------'


 5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  3  2  1  0
 5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  3  2  1  0
 5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  3  2  1  0
 5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  3  2  1  0
 5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  3  2  1  0
 5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  4  3  2  1  0
 5  5  5  5  5  5  5  5  5  5  5  5  5  5  4  4  3  3  2  1  0
 5  5  5  5  5  5  5  5  5  5  5  5  5  4  4  3  3  2  2  1  0
 5  5  5  5  5  5  5  5  5  5  5  4  4  4  3  3  2  2  1  1  0
 5  5  5  5  5  5  5  5  4  4  4  4  3  3  3  2  2  1  1  0  0
 5  5  5  5  5  4  4  4  4  3  3  3  3  2  2  2  1  1  0  0  0
 5  5  5  4  4  4  3  3  3  3  2  2  2  2  1  1  1  0  0  0  0
 5  4  4  4  3  3  3  2  2  2  2  1  1  1  1  0  0  0  0  0  0
 4  4  3  3  3  2  2  2  1  1  1  1  0  0  0  0  0  0  0 -1 -1
 3  3  3  2  2  2  1  1  1  0  0  0  0  0  0  0  0 -1 -1 -1 -2
 3  2  2  2  1  1  1  0  0  0  0  0  0  0 -1 -1 -1 -1 -